In [2]:
import numpy as np
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
import scanpy as sc
import pandas as pd
import torch
import scipy
import time
from model import *
from utils import *

In [4]:
scdata = pd.read_csv('/home/ehtesamul/sc_st/data/mousedata_2020/E1z2/simu_sc_counts.csv', index_col = 0)
scdata.head(5)

,embryo1_Pos0_cell100_z2,embryo1_Pos0_cell101_z2,embryo1_Pos0_cell102_z2,embryo1_Pos0_cell105_z2,embryo1_Pos0_cell106_z2,embryo1_Pos0_cell107_z2,embryo1_Pos0_cell108_z2,embryo1_Pos0_cell109_z2,embryo1_Pos0_cell11_z2,embryo1_Pos0_cell110_z2,...,embryo1_Pos28_cell9_z2,embryo1_Pos28_cell90_z2,embryo1_Pos28_cell91_z2,embryo1_Pos28_cell92_z2,embryo1_Pos28_cell93_z2,embryo1_Pos28_cell94_z2,embryo1_Pos28_cell95_z2,embryo1_Pos28_cell96_z2,embryo1_Pos28_cell97_z2,embryo1_Pos28_cell98_z2
Abcc4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,2,0,0,0,2
Acp5,2,1,0,2,1,1,0,0,0,0,...,0,0,1,0,2,0,0,1,0,2
Acvr1,0,0,1,0,1,1,0,0,0,0,...,1,1,0,0,0,0,1,0,0,0
Acvr2a,0,1,0,1,0,0,0,0,1,0,...,3,2,0,0,0,0,2,0,0,2
Adora2b,0,0,1,0,0,0,0,0,1,0,...,2,0,0,0,0,0,0,0,0,0


In [5]:
scdata.shape

(351, 10150)

In [6]:
stdata = pd.read_csv('/home/ehtesamul/sc_st/data/mousedata_2020/E1z2/simu_st_counts.csv', index_col = 0)
stdata.head(5)

,ST_12,ST_13,ST_42,ST_43,ST_49,ST_50,ST_51,ST_52,ST_53,ST_72,...,ST_1158,ST_1159,ST_1179,ST_1180,ST_1181,ST_1182,ST_1183,ST_1184,ST_1185,ST_1186
Abcc4,0.0,0.0,5.0,0.0,1.0,1.0,4.0,1.0,1.0,3.0,...,1.0,0.0,1.0,2.0,0.0,3.0,1.0,2.0,3.0,0.0
Acp5,1.0,1.0,5.0,0.0,5.0,0.0,4.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2.0,0.0,1.0,0.0,1.0
Acvr1,0.0,2.0,2.0,0.0,1.0,3.0,5.0,7.0,2.0,1.0,...,1.0,1.0,2.0,0.0,1.0,0.0,0.0,0.0,1.0,3.0
Acvr2a,2.0,4.0,11.0,0.0,1.0,2.0,1.0,2.0,1.0,3.0,...,2.0,1.0,0.0,0.0,2.0,0.0,1.0,1.0,1.0,1.0
Adora2b,1.0,2.0,2.0,0.0,0.0,1.0,0.0,4.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0


In [21]:
stdata.shape

(351, 582)

In [7]:
scdata = scdata.T
stdata = stdata.T

In [8]:
stgtcelltype = pd.read_csv('/home/ehtesamul/sc_st/data/mousedata_2020/E1z2/simu_st_celltype.csv', index_col = 0)
stgtcelltype.head(5)

,Allantois,Anterior somitic tissues,Blood progenitors,Cardiomyocytes,Cranial mesoderm,Definitive endoderm,Dermomyotome,Endothelium,Erythroid,ExE endoderm,...,Intermediate mesoderm,Lateral plate mesoderm,Mixed mesenchymal mesoderm,NMP,Neural crest,Presomitic mesoderm,Sclerotome,Spinal cord,Splanchnic mesoderm,Surface ectoderm
ST_12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
ST_13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0
ST_42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
ST_43,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
ST_49,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.666667,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [ ]:
stgtcelltype.shape

(581, 23)

In [9]:
scdata.head(5)

,Abcc4,Acp5,Acvr1,Acvr2a,Adora2b,Afp,Ahnak,Akr1c19,Alas2,Aldh1a2,...,Wnt2b,Wnt3,Wnt3a,Wnt5a,Wnt5b,Wnt8a,Xist,Zfp444,Zfp57,Zic3
embryo1_Pos0_cell100_z2,0,2,0,0,0,2,0,0,2,0,...,0,0,1,0,0,1,1,0,0,1
embryo1_Pos0_cell101_z2,0,1,0,1,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
embryo1_Pos0_cell102_z2,0,0,1,0,1,0,6,6,1,2,...,0,0,0,11,1,1,19,3,1,0
embryo1_Pos0_cell105_z2,0,2,0,1,0,0,0,0,2,0,...,0,0,0,1,1,0,0,0,1,0
embryo1_Pos0_cell106_z2,0,1,1,0,0,0,1,3,2,0,...,0,0,2,5,0,0,2,3,1,0


In [10]:
stdata.head(5)

,Abcc4,Acp5,Acvr1,Acvr2a,Adora2b,Afp,Ahnak,Akr1c19,Alas2,Aldh1a2,...,Wnt2b,Wnt3,Wnt3a,Wnt5a,Wnt5b,Wnt8a,Xist,Zfp444,Zfp57,Zic3
ST_12,0.0,1.0,0.0,2.0,1.0,1.0,0.0,2.0,2.0,1.0,...,0.0,1.0,3.0,2.0,1.0,0.0,3.0,3.0,4.0,1.0
ST_13,0.0,1.0,2.0,4.0,2.0,0.0,0.0,4.0,2.0,2.0,...,0.0,1.0,1.0,1.0,0.0,0.0,3.0,0.0,6.0,2.0
ST_42,5.0,5.0,2.0,11.0,2.0,1.0,1.0,11.0,3.0,2.0,...,1.0,0.0,7.0,6.0,1.0,1.0,13.0,8.0,7.0,3.0
ST_43,0.0,0.0,0.0,0.0,0.0,1.0,4.0,3.0,0.0,0.0,...,0.0,1.0,2.0,0.0,0.0,1.0,0.0,3.0,0.0,1.0
ST_49,1.0,5.0,1.0,1.0,0.0,1.0,2.0,2.0,9.0,0.0,...,0.0,0.0,1.0,2.0,0.0,1.0,2.0,0.0,1.0,0.0


In [11]:
stdata.shape

(581, 351)

In [12]:
spcoor = pd.read_csv('/home/ehtesamul/sc_st/data/mousedata_2020/E1z2/simu_st_metadata.csv', index_col=0)
spcoor.head(5)

,coord_x,coord_y
ST_12,1581.242732,1329.313864
ST_13,1586.662026,1329.313864
ST_42,1581.242732,1334.863919
ST_43,1586.662026,1334.863919
ST_49,1619.177788,1334.863919


In [13]:
spcoor.shape

(581, 2)

In [14]:
scmetadata = pd.read_csv('/home/ehtesamul/sc_st/data/mousedata_2020/E1z2/metadata.csv', index_col = 0)
scmetadata.head(5)

,orig.ident,nCount_RNA,nFeature_RNA,uniqueID,embryo,pos,z,x_global,y_global,x_global_affine,y_global_affine,embryo_pos,embryo_pos_z,Area,UMAP1,UMAP2,celltype_mapped_refined
embryo1_Pos0_cell100_z2,embryo1,107,80,embryo1_Pos0_cell100_z2,embryo1,Pos0,2,1625.055128,1346.268007,0.961726,-2.943951,embryo1_Pos0,embryo1_Pos0_2,8144,-6.462337,-1.034258,Erythroid
embryo1_Pos0_cell101_z2,embryo1,106,70,embryo1_Pos0_cell101_z2,embryo1,Pos0,2,1624.997889,1344.508417,0.959880,-3.000712,embryo1_Pos0,embryo1_Pos0_2,8560,-6.810907,-0.845417,Erythroid
embryo1_Pos0_cell102_z2,embryo1,886,210,embryo1_Pos0_cell102_z2,embryo1,Pos0,2,1625.716288,1335.714958,0.983054,-3.284372,embryo1_Pos0,embryo1_Pos0_2,15936,4.067604,0.122410,Gut tube
embryo1_Pos0_cell105_z2,embryo1,99,68,embryo1_Pos0_cell105_z2,embryo1,Pos0,2,1625.282523,1349.823455,0.969061,-2.829259,embryo1_Pos0,embryo1_Pos0_2,6768,-6.670613,-0.926943,Erythroid
embryo1_Pos0_cell106_z2,embryo1,267,147,embryo1_Pos0_cell106_z2,embryo1,Pos0,2,1625.242317,1352.717525,0.967764,-2.735902,embryo1_Pos0,embryo1_Pos0_2,7776,-3.569233,-3.798838,Haematoendothelial progenitors


In [15]:
scmetadata.shape

(10150, 17)

In [16]:
adata = sc.AnnData(scdata, obs=scmetadata)
adata

AnnData object with n_obs × n_vars = 10150 × 351
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'uniqueID', 'embryo', 'pos', 'z', 'x_global', 'y_global', 'x_global_affine', 'y_global_affine', 'embryo_pos', 'embryo_pos_z', 'Area', 'UMAP1', 'UMAP2', 'celltype_mapped_refined'

In [17]:
adata.X

array([[0, 2, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 3, 1, 0],
       ...,
       [0, 1, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 3, 0, 0],
       [2, 2, 0, ..., 1, 3, 1]])

In [18]:
adata.var_names

Index(['Abcc4', 'Acp5', 'Acvr1', 'Acvr2a', 'Adora2b', 'Afp', 'Ahnak',
       'Akr1c19', 'Alas2', 'Aldh1a2',
       ...
       'Wnt2b', 'Wnt3', 'Wnt3a', 'Wnt5a', 'Wnt5b', 'Wnt8a', 'Xist', 'Zfp444',
       'Zfp57', 'Zic3'],
      dtype='object', length=351)

In [19]:
adata.obs_names

Index(['embryo1_Pos0_cell100_z2', 'embryo1_Pos0_cell101_z2',
       'embryo1_Pos0_cell102_z2', 'embryo1_Pos0_cell105_z2',
       'embryo1_Pos0_cell106_z2', 'embryo1_Pos0_cell107_z2',
       'embryo1_Pos0_cell108_z2', 'embryo1_Pos0_cell109_z2',
       'embryo1_Pos0_cell11_z2', 'embryo1_Pos0_cell110_z2',
       ...
       'embryo1_Pos28_cell9_z2', 'embryo1_Pos28_cell90_z2',
       'embryo1_Pos28_cell91_z2', 'embryo1_Pos28_cell92_z2',
       'embryo1_Pos28_cell93_z2', 'embryo1_Pos28_cell94_z2',
       'embryo1_Pos28_cell95_z2', 'embryo1_Pos28_cell96_z2',
       'embryo1_Pos28_cell97_z2', 'embryo1_Pos28_cell98_z2'],
      dtype='object', length=10150)

In [20]:
scdata.head(5)

,Abcc4,Acp5,Acvr1,Acvr2a,Adora2b,Afp,Ahnak,Akr1c19,Alas2,Aldh1a2,...,Wnt2b,Wnt3,Wnt3a,Wnt5a,Wnt5b,Wnt8a,Xist,Zfp444,Zfp57,Zic3
embryo1_Pos0_cell100_z2,0,2,0,0,0,2,0,0,2,0,...,0,0,1,0,0,1,1,0,0,1
embryo1_Pos0_cell101_z2,0,1,0,1,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
embryo1_Pos0_cell102_z2,0,0,1,0,1,0,6,6,1,2,...,0,0,0,11,1,1,19,3,1,0
embryo1_Pos0_cell105_z2,0,2,0,1,0,0,0,0,2,0,...,0,0,0,1,1,0,0,0,1,0
embryo1_Pos0_cell106_z2,0,1,1,0,0,0,1,3,2,0,...,0,0,2,5,0,0,2,3,1,0


In [21]:
scmetadata

,orig.ident,nCount_RNA,nFeature_RNA,uniqueID,embryo,pos,z,x_global,y_global,x_global_affine,y_global_affine,embryo_pos,embryo_pos_z,Area,UMAP1,UMAP2,celltype_mapped_refined
embryo1_Pos0_cell100_z2,embryo1,107,80,embryo1_Pos0_cell100_z2,embryo1,Pos0,2,1625.055128,1346.268007,0.961726,-2.943951,embryo1_Pos0,embryo1_Pos0_2,8144,-6.462337,-1.034258,Erythroid
embryo1_Pos0_cell101_z2,embryo1,106,70,embryo1_Pos0_cell101_z2,embryo1,Pos0,2,1624.997889,1344.508417,0.959880,-3.000712,embryo1_Pos0,embryo1_Pos0_2,8560,-6.810907,-0.845417,Erythroid
embryo1_Pos0_cell102_z2,embryo1,886,210,embryo1_Pos0_cell102_z2,embryo1,Pos0,2,1625.716288,1335.714958,0.983054,-3.284372,embryo1_Pos0,embryo1_Pos0_2,15936,4.067604,0.122410,Gut tube
embryo1_Pos0_cell105_z2,embryo1,99,68,embryo1_Pos0_cell105_z2,embryo1,Pos0,2,1625.282523,1349.823455,0.969061,-2.829259,embryo1_Pos0,embryo1_Pos0_2,6768,-6.670613,-0.926943,Erythroid
embryo1_Pos0_cell106_z2,embryo1,267,147,embryo1_Pos0_cell106_z2,embryo1,Pos0,2,1625.242317,1352.717525,0.967764,-2.735902,embryo1_Pos0,embryo1_Pos0_2,7776,-3.569233,-3.798838,Haematoendothelial progenitors
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
embryo1_Pos28_cell94_z2,embryo1,249,117,embryo1_Pos28_cell94_z2,embryo1,Pos28,2,1547.532898,1518.029214,-1.538991,2.596733,embryo1_Pos28,embryo1_Pos28_2,8160,1.387528,3.173942,Spinal cord
embryo1_Pos28_cell95_z2,embryo1,385,152,embryo1_Pos28_cell95_z2,embryo1,Pos28,2,1547.675803,1528.829589,-1.534381,2.945132,embryo1_Pos28,embryo1_Pos28_2,10032,-1.776725,-1.773125,Anterior somitic tissues
embryo1_Pos28_cell96_z2,embryo1,264,133,embryo1_Pos28_cell96_z2,embryo1,Pos28,2,1547.365765,1535.121229,-1.544382,3.148089,embryo1_Pos28,embryo1_Pos28_2,8368,0.336822,-0.025109,Neural crest
embryo1_Pos28_cell97_z2,embryo1,355,152,embryo1_Pos28_cell97_z2,embryo1,Pos28,2,1547.535748,1520.121983,-1.538899,2.664242,embryo1_Pos28,embryo1_Pos28_2,7552,1.500579,4.424931,Spinal cord


In [22]:
new_scdata = pd.DataFrame(adata.X,index=adata.obs_names,columns=adata.var_names)
new_scdata.head(5)

,Abcc4,Acp5,Acvr1,Acvr2a,Adora2b,Afp,Ahnak,Akr1c19,Alas2,Aldh1a2,...,Wnt2b,Wnt3,Wnt3a,Wnt5a,Wnt5b,Wnt8a,Xist,Zfp444,Zfp57,Zic3
embryo1_Pos0_cell100_z2,0,2,0,0,0,2,0,0,2,0,...,0,0,1,0,0,1,1,0,0,1
embryo1_Pos0_cell101_z2,0,1,0,1,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
embryo1_Pos0_cell102_z2,0,0,1,0,1,0,6,6,1,2,...,0,0,0,11,1,1,19,3,1,0
embryo1_Pos0_cell105_z2,0,2,0,1,0,0,0,0,2,0,...,0,0,0,1,1,0,0,0,1,0
embryo1_Pos0_cell106_z2,0,1,1,0,0,0,1,3,2,0,...,0,0,2,5,0,0,2,3,1,0


In [23]:
adata.obs['embryo']

embryo1_Pos0_cell100_z2    embryo1
embryo1_Pos0_cell101_z2    embryo1
embryo1_Pos0_cell102_z2    embryo1
embryo1_Pos0_cell105_z2    embryo1
embryo1_Pos0_cell106_z2    embryo1
                            ...   
embryo1_Pos28_cell94_z2    embryo1
embryo1_Pos28_cell95_z2    embryo1
embryo1_Pos28_cell96_z2    embryo1
embryo1_Pos28_cell97_z2    embryo1
embryo1_Pos28_cell98_z2    embryo1
Name: embryo, Length: 10150, dtype: object

In [28]:
scdata.head(5)

,Abcc4,Acp5,Acvr1,Acvr2a,Adora2b,Afp,Ahnak,Akr1c19,Alas2,Aldh1a2,...,Wnt2b,Wnt3,Wnt3a,Wnt5a,Wnt5b,Wnt8a,Xist,Zfp444,Zfp57,Zic3
embryo1_Pos0_cell100_z2,0,2,0,0,0,2,0,0,2,0,...,0,0,1,0,0,1,1,0,0,1
embryo1_Pos0_cell101_z2,0,1,0,1,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
embryo1_Pos0_cell102_z2,0,0,1,0,1,0,6,6,1,2,...,0,0,0,11,1,1,19,3,1,0
embryo1_Pos0_cell105_z2,0,2,0,1,0,0,0,0,2,0,...,0,0,0,1,1,0,0,0,1,0
embryo1_Pos0_cell106_z2,0,1,1,0,0,0,1,3,2,0,...,0,0,2,5,0,0,2,3,1,0


In [29]:
new_scdata.head(5)

,Abcc4,Acp5,Acvr1,Acvr2a,Adora2b,Afp,Ahnak,Akr1c19,Alas2,Aldh1a2,...,Wnt2b,Wnt3,Wnt3a,Wnt5a,Wnt5b,Wnt8a,Xist,Zfp444,Zfp57,Zic3
embryo1_Pos0_cell100_z2,0,2,0,0,0,2,0,0,2,0,...,0,0,1,0,0,1,1,0,0,1
embryo1_Pos0_cell101_z2,0,1,0,1,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
embryo1_Pos0_cell102_z2,0,0,1,0,1,0,6,6,1,2,...,0,0,0,11,1,1,19,3,1,0
embryo1_Pos0_cell105_z2,0,2,0,1,0,0,0,0,2,0,...,0,0,0,1,1,0,0,0,1,0
embryo1_Pos0_cell106_z2,0,1,1,0,0,0,1,3,2,0,...,0,0,2,5,0,0,2,3,1,0


In [30]:
adata.obs

,orig.ident,nCount_RNA,nFeature_RNA,uniqueID,embryo,pos,z,x_global,y_global,x_global_affine,y_global_affine,embryo_pos,embryo_pos_z,Area,UMAP1,UMAP2,celltype_mapped_refined
embryo1_Pos0_cell100_z2,embryo1,107,80,embryo1_Pos0_cell100_z2,embryo1,Pos0,2,1625.055128,1346.268007,0.961726,-2.943951,embryo1_Pos0,embryo1_Pos0_2,8144,-6.462337,-1.034258,Erythroid
embryo1_Pos0_cell101_z2,embryo1,106,70,embryo1_Pos0_cell101_z2,embryo1,Pos0,2,1624.997889,1344.508417,0.959880,-3.000712,embryo1_Pos0,embryo1_Pos0_2,8560,-6.810907,-0.845417,Erythroid
embryo1_Pos0_cell102_z2,embryo1,886,210,embryo1_Pos0_cell102_z2,embryo1,Pos0,2,1625.716288,1335.714958,0.983054,-3.284372,embryo1_Pos0,embryo1_Pos0_2,15936,4.067604,0.122410,Gut tube
embryo1_Pos0_cell105_z2,embryo1,99,68,embryo1_Pos0_cell105_z2,embryo1,Pos0,2,1625.282523,1349.823455,0.969061,-2.829259,embryo1_Pos0,embryo1_Pos0_2,6768,-6.670613,-0.926943,Erythroid
embryo1_Pos0_cell106_z2,embryo1,267,147,embryo1_Pos0_cell106_z2,embryo1,Pos0,2,1625.242317,1352.717525,0.967764,-2.735902,embryo1_Pos0,embryo1_Pos0_2,7776,-3.569233,-3.798838,Haematoendothelial progenitors
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
embryo1_Pos28_cell94_z2,embryo1,249,117,embryo1_Pos28_cell94_z2,embryo1,Pos28,2,1547.532898,1518.029214,-1.538991,2.596733,embryo1_Pos28,embryo1_Pos28_2,8160,1.387528,3.173942,Spinal cord
embryo1_Pos28_cell95_z2,embryo1,385,152,embryo1_Pos28_cell95_z2,embryo1,Pos28,2,1547.675803,1528.829589,-1.534381,2.945132,embryo1_Pos28,embryo1_Pos28_2,10032,-1.776725,-1.773125,Anterior somitic tissues
embryo1_Pos28_cell96_z2,embryo1,264,133,embryo1_Pos28_cell96_z2,embryo1,Pos28,2,1547.365765,1535.121229,-1.544382,3.148089,embryo1_Pos28,embryo1_Pos28_2,8368,0.336822,-0.025109,Neural crest
embryo1_Pos28_cell97_z2,embryo1,355,152,embryo1_Pos28_cell97_z2,embryo1,Pos28,2,1547.535748,1520.121983,-1.538899,2.664242,embryo1_Pos28,embryo1_Pos28_2,7552,1.500579,4.424931,Spinal cord


In [31]:
scdata = new_scdata
stadata = sc.AnnData(stdata)
sc.pp.normalize_total(stadata)
sc.pp.log1p(stadata)
stdata = pd.DataFrame(stadata.X, index = stadata.obs_names, columns = stadata.var_names)

In [33]:
adata.obsm['spatial'] = scmetadata[['x_global', 'y_global']].values

In [34]:
adata.obs

,orig.ident,nCount_RNA,nFeature_RNA,uniqueID,embryo,pos,z,x_global,y_global,x_global_affine,y_global_affine,embryo_pos,embryo_pos_z,Area,UMAP1,UMAP2,celltype_mapped_refined
embryo1_Pos0_cell100_z2,embryo1,107,80,embryo1_Pos0_cell100_z2,embryo1,Pos0,2,1625.055128,1346.268007,0.961726,-2.943951,embryo1_Pos0,embryo1_Pos0_2,8144,-6.462337,-1.034258,Erythroid
embryo1_Pos0_cell101_z2,embryo1,106,70,embryo1_Pos0_cell101_z2,embryo1,Pos0,2,1624.997889,1344.508417,0.959880,-3.000712,embryo1_Pos0,embryo1_Pos0_2,8560,-6.810907,-0.845417,Erythroid
embryo1_Pos0_cell102_z2,embryo1,886,210,embryo1_Pos0_cell102_z2,embryo1,Pos0,2,1625.716288,1335.714958,0.983054,-3.284372,embryo1_Pos0,embryo1_Pos0_2,15936,4.067604,0.122410,Gut tube
embryo1_Pos0_cell105_z2,embryo1,99,68,embryo1_Pos0_cell105_z2,embryo1,Pos0,2,1625.282523,1349.823455,0.969061,-2.829259,embryo1_Pos0,embryo1_Pos0_2,6768,-6.670613,-0.926943,Erythroid
embryo1_Pos0_cell106_z2,embryo1,267,147,embryo1_Pos0_cell106_z2,embryo1,Pos0,2,1625.242317,1352.717525,0.967764,-2.735902,embryo1_Pos0,embryo1_Pos0_2,7776,-3.569233,-3.798838,Haematoendothelial progenitors
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
embryo1_Pos28_cell94_z2,embryo1,249,117,embryo1_Pos28_cell94_z2,embryo1,Pos28,2,1547.532898,1518.029214,-1.538991,2.596733,embryo1_Pos28,embryo1_Pos28_2,8160,1.387528,3.173942,Spinal cord
embryo1_Pos28_cell95_z2,embryo1,385,152,embryo1_Pos28_cell95_z2,embryo1,Pos28,2,1547.675803,1528.829589,-1.534381,2.945132,embryo1_Pos28,embryo1_Pos28_2,10032,-1.776725,-1.773125,Anterior somitic tissues
embryo1_Pos28_cell96_z2,embryo1,264,133,embryo1_Pos28_cell96_z2,embryo1,Pos28,2,1547.365765,1535.121229,-1.544382,3.148089,embryo1_Pos28,embryo1_Pos28_2,8368,0.336822,-0.025109,Neural crest
embryo1_Pos28_cell97_z2,embryo1,355,152,embryo1_Pos28_cell97_z2,embryo1,Pos28,2,1547.535748,1520.121983,-1.538899,2.664242,embryo1_Pos28,embryo1_Pos28_2,7552,1.500579,4.424931,Spinal cord


In [35]:
spcoor

,coord_x,coord_y
ST_12,1581.242732,1329.313864
ST_13,1586.662026,1329.313864
ST_42,1581.242732,1334.863919
ST_43,1586.662026,1334.863919
ST_49,1619.177788,1334.863919
...,...,...
ST_1182,1581.242732,1545.766004
ST_1183,1586.662026,1545.766004
ST_1184,1592.081319,1545.766004
ST_1185,1597.500613,1545.766004


In [37]:
adata.obsm['spatial']

array([[1625.0551283 , 1346.26800741],
       [1624.99788853, 1344.50841715],
       [1625.716288  , 1335.71495847],
       ...,
       [1547.36576514, 1535.12122866],
       [1547.53574798, 1520.12198341],
       [1548.30892522, 1518.91970479]])